# InfluxDB with Python 

* https://influxdb-python.readthedocs.io/en/latest/examples.html

## Create and Show

* https://www.influxdata.com/blog/getting-started-python-influxdb/

In [1]:
from influxdb import InfluxDBClient

In [2]:
# client = InfluxDBClient(host='localhost', port=8086)
client = InfluxDBClient(host='localhost', port=8086, 
                        username='admin', password='admin')  #ssl=True, verify_ssl=True)

In [3]:
database_name = 'python-influxdb'
client.create_database(database_name)

In [4]:
json_body = [
    {
        "measurement": "cpu_load_short",
        "tags": {
            "host": "server01",
            "region": "us-west"
        },
        "time": "2009-11-10T23:00:00Z",
        "fields": {
            "value": 0.64
        }
    }
]

client.write_points(json_body, database=database_name)

True

In [5]:
rs = client.query('select value from cpu_load_short;', database=database_name)
rs

ResultSet({'('cpu_load_short', None)': [{'time': '2009-11-10T23:00:00Z', 'value': 0.64}]})

## Queries

### Show Databases 

In [6]:
client = InfluxDBClient(host='localhost', port=8086, username='admin', password='admin')
rs = client.query('show databases;')
rs

ResultSet({'('databases', None)': [{'name': 'tick_experiment'}, {'name': '_internal'}, {'name': 'python-influxdb'}, {'name': 'NOAA_water_database'}]})

In [7]:
client.get_list_database()

[{'name': 'tick_experiment'},
 {'name': '_internal'},
 {'name': 'python-influxdb'},
 {'name': 'NOAA_water_database'}]

### Switch Database

In [8]:
client.switch_database('NOAA_water_database')

In [9]:
client.query('show measurements;')

ResultSet({'('measurements', None)': [{'name': 'average_temperature'}, {'name': 'h2o_feet'}, {'name': 'h2o_pH'}, {'name': 'h2o_quality'}, {'name': 'h2o_temperature'}]})

### Get Point

* https://influxdb-python.readthedocs.io/en/latest/resultset.html

In [10]:
rs = client.query('select * from average_temperature;')

In [11]:
cpu_points = list(rs.get_points())
cpu_points

[{'time': '2015-08-18T00:00:00Z', 'degrees': 82, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:00:00Z', 'degrees': 85, 'location': 'santa_monica'},
 {'time': '2015-08-18T00:06:00Z', 'degrees': 73, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:06:00Z', 'degrees': 74, 'location': 'santa_monica'},
 {'time': '2015-08-18T00:12:00Z', 'degrees': 86, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:12:00Z', 'degrees': 80, 'location': 'santa_monica'},
 {'time': '2015-08-18T00:18:00Z', 'degrees': 89, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:18:00Z', 'degrees': 81, 'location': 'santa_monica'},
 {'time': '2015-08-18T00:24:00Z', 'degrees': 77, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:24:00Z', 'degrees': 81, 'location': 'santa_monica'},
 {'time': '2015-08-18T00:30:00Z', 'degrees': 70, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:30:00Z', 'degrees': 71, 'location': 'santa_monica'},
 {'time': '2015-08-18T00:36:00Z', 'degrees': 84, 'location': 'co

In [12]:
cpu_points = list(rs.get_points(measurement='average_temperature',
                               tags={'location': 'coyote_creek'}))
cpu_points

[{'time': '2015-08-18T00:00:00Z', 'degrees': 82, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:06:00Z', 'degrees': 73, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:12:00Z', 'degrees': 86, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:18:00Z', 'degrees': 89, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:24:00Z', 'degrees': 77, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:30:00Z', 'degrees': 70, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:36:00Z', 'degrees': 84, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:42:00Z', 'degrees': 76, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:48:00Z', 'degrees': 85, 'location': 'coyote_creek'},
 {'time': '2015-08-18T00:54:00Z', 'degrees': 80, 'location': 'coyote_creek'},
 {'time': '2015-08-18T01:00:00Z', 'degrees': 70, 'location': 'coyote_creek'},
 {'time': '2015-08-18T01:06:00Z', 'degrees': 77, 'location': 'coyote_creek'},
 {'time': '2015-08-18T01:12:00Z', 'degrees': 90, 'location': 'co

### Querying Data 

* https://github.com/influxdata/docs.influxdata.com/blob/master/content/influxdb/v1.1/troubleshooting/errors.md#invalid-operation-time-and-influxqlvarref-are-not-compatible

In [13]:
# rs = client.query('select * from h2o_quality where time <= \'2015-08-19T00:00:00Z\' group by "location"')
rs = client.query('select * from NOAA_water_database.autogen.h2o_quality where time <= \'2015-08-19T00:00:00Z\' group by "location"')
list(rs.get_points())

[{'time': '2015-08-18T00:00:00Z', 'index': 41, 'randtag': '1'},
 {'time': '2015-08-18T00:06:00Z', 'index': 11, 'randtag': '3'},
 {'time': '2015-08-18T00:12:00Z', 'index': 38, 'randtag': '1'},
 {'time': '2015-08-18T00:18:00Z', 'index': 50, 'randtag': '1'},
 {'time': '2015-08-18T00:24:00Z', 'index': 35, 'randtag': '3'},
 {'time': '2015-08-18T00:30:00Z', 'index': 49, 'randtag': '2'},
 {'time': '2015-08-18T00:36:00Z', 'index': 24, 'randtag': '1'},
 {'time': '2015-08-18T00:42:00Z', 'index': 92, 'randtag': '3'},
 {'time': '2015-08-18T00:48:00Z', 'index': 56, 'randtag': '3'},
 {'time': '2015-08-18T00:54:00Z', 'index': 32, 'randtag': '2'},
 {'time': '2015-08-18T01:00:00Z', 'index': 68, 'randtag': '3'},
 {'time': '2015-08-18T01:06:00Z', 'index': 50, 'randtag': '2'},
 {'time': '2015-08-18T01:12:00Z', 'index': 94, 'randtag': '1'},
 {'time': '2015-08-18T01:18:00Z', 'index': 11, 'randtag': '3'},
 {'time': '2015-08-18T01:24:00Z', 'index': 90, 'randtag': '2'},
 {'time': '2015-08-18T01:30:00Z', 'index

## PANDAS

* https://github.com/influxdata/influxdb-python/blob/master/examples/tutorial_pandas.py

In [14]:
import argparse
import pandas as pd
from influxdb import DataFrameClient

In [15]:
host = 'localhost'
port = 8086

# Instantiate the connection to the InfluxDB client.
user     = 'admin'
password = 'admin'
dbname   = 'try-pandas'
protocol = 'json'
measurement = 'demo'

In [16]:
client = DataFrameClient(host=host, port=port, username=user, password=password, database=dbname)

In [17]:
# Create pandas DataFrame
df = pd.DataFrame(data=list(range(30)), 
                  index=pd.date_range(start='2014-11-16', periods=30, freq='H'),
                  columns=['0'])

In [18]:
# Create database
# Here I got error on 5.2.0 version of python-influxdb, please be aware.
client.drop_database(dbname)
client.create_database(dbname)

In [19]:
# Write DataFrame
client.write_points(df, measurement, protocol=protocol)
# Write DataFrame with Tags
client.write_points(df, measurement,tags={'k1': 'v1', 'k2': 'v2'}, protocol=protocol)
# Read DataFrame back
rs = client.query('select * from {}'.format(measurement))

# Delete database
client.drop_database(dbname)

In [20]:
rs

defaultdict(list, {'demo':                             0    k1    k2
             2014-11-16 00:00:00+00:00   0  None  None
             2014-11-16 00:00:00+00:00   0    v1    v2
             2014-11-16 01:00:00+00:00   1  None  None
             2014-11-16 01:00:00+00:00   1    v1    v2
             2014-11-16 02:00:00+00:00   2  None  None
             2014-11-16 02:00:00+00:00   2    v1    v2
             2014-11-16 03:00:00+00:00   3  None  None
             2014-11-16 03:00:00+00:00   3    v1    v2
             2014-11-16 04:00:00+00:00   4  None  None
             2014-11-16 04:00:00+00:00   4    v1    v2
             2014-11-16 05:00:00+00:00   5  None  None
             2014-11-16 05:00:00+00:00   5    v1    v2
             2014-11-16 06:00:00+00:00   6  None  None
             2014-11-16 06:00:00+00:00   6    v1    v2
             2014-11-16 07:00:00+00:00   7  None  None
             2014-11-16 07:00:00+00:00   7    v1    v2
             2014-11-16 08:00:00+00:00   8  None  N

---